## agentを利用したアプリケーション

In [84]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [53]:
from dotenv import load_dotenv

load_dotenv("../.env")

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, CohereEmbeddings(model="embed-multilingual-v3.0"))
retriever = vector.as_retriever()

retriever.invoke("how to upload a dataset")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='"revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {  description: "A sample dataset in LangSmith.",});await client.createExamples({  inputs: [    { postfix: "to LangSmith" },    { postfix: "to Evaluations in LangSmith" },  ],  outputs: [    { output: "Welcome to LangSmith" },  

In [77]:
from langchain_core.tools import tool
from enum import Enum

# 曜日のEnum定義
class DayOfWeek(Enum):
    MONDAY = 'Monday'
    TUESDAY = 'Tuesday'
    WEDNESDAY = 'Wednesday'
    THURSDAY = 'Thursday'
    FRIDAY = 'Friday'
    SATURDAY = 'Saturday'
    SUNDAY = 'Sunday'

day_values_map = {
    DayOfWeek.MONDAY: 0,
    DayOfWeek.TUESDAY: 10,
    DayOfWeek.WEDNESDAY: 20,
    DayOfWeek.THURSDAY: 30,
    DayOfWeek.FRIDAY: 40,
    DayOfWeek.SATURDAY: 50,
    DayOfWeek.SUNDAY: 100,
}

@tool
def combat_power(day: DayOfWeek) -> int:
    """
    曜日に対応するBigBaByの戦闘力を計算します。

    Args:
      day: 曜日です。
    """

    return day_values_map.get(day)

print(combat_power.name)
print(combat_power.description)
print(combat_power.args)


combat_power
曜日に対応するBigBaByの戦闘力を計算します。

Args:
  day: 曜日です。
{'day': {'$ref': '#/$defs/DayOfWeek'}}


In [78]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
tools = [combat_power, retriever_tool]
tools

[StructuredTool(name='combat_power', description='曜日に対応するBigBaByの戦闘力を計算します。\n\nArgs:\n  day: 曜日です。', args_schema=<class 'langchain_core.utils.pydantic.combat_power'>, func=<function combat_power at 0x7fb2311b6200>),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7fb24bfa7e20>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fb231196600>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7fb24bce49a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<

In [79]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke("おはよう")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: おはようございます！今日も一日頑張りましょう！
ToolCalls: []


In [83]:
response = model_with_tools.invoke("BigBaByの月曜の次の日の戦闘力は？")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I will ask for the combat power of Tuesday and share the result with the user.
ToolCalls: [{'name': 'combat_power', 'args': {'day': 'Tuesday'}, 'id': 'f08970f7019a4f86aa18f3fa25bf1e88', 'type': 'tool_call'}]


In [87]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer

system_message = "You are a helpful assistant."
memory = MemorySaver()
langgraph_agent_executor = create_react_agent(
  model, tools,state_modifier=system_message, checkpointer=memory
)
config = {"configurable": {"thread_id": "test-thread"}}
langgraph_agent_executor.invoke(
  {
    "messages": [
      ("user","BigBaByの木曜の戦闘力は？"),
    ]
  },
  config,
)

{'messages': [HumanMessage(content='BigBaByの木曜の戦闘力は？', additional_kwargs={}, response_metadata={}, id='e6c552bb-4efd-4f2a-bfae-60ae7f727b83'),
  AIMessage(content="I will run a search to find out what BigBaBy's combat power is on Thursdays.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2aadfdc0-98a7-4d6c-88aa-86504b14b627', 'tool_calls': [{'id': '52b5b223ce31411a99d1ae7ec5645d47', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thursday"}'}, 'type': 'function'}], 'token_count': {'input_tokens': 986.0, 'output_tokens': 61.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2aadfdc0-98a7-4d6c-88aa-86504b14b627', 'tool_calls': [{'id': '52b5b223ce31411a99d1ae7ec5645d47', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thursday"}'}, 'type': 'function'}], 'toke